# Specifying a wave function in Forte

To uniquely identify an electronic state, Forte needs to know the total number of
electrons ($N$), the number of alpha and beta electrons
($N_{\alpha}/N_{\beta}$), the spin state, and the symmetry of the state(s) computed.
These quantities are specified via various options, as detailed below.

## Charge and spin multiplicity

The molecular charge ($Q$), is defined as the sum of the
charge of the nuclei ($Z_A$) minus the number of electrons

$$
Q = \sum_A Z_A - N_\text{el}.
$$

Recall that, in the absence of an external field, an electronic state is
an eigenfunction of the spin squared operator $\hat{S}^2$ and 
the z component of the spin operator $\hat{S}_z$.
The eigenvalues of these two operators are related to the spin quantum
numbers $S$ and $M_S$ via (in atomic units)

$${\hat{S}}^2 |S,M_S\rangle = S(S+1)|S,M_S\rangle,$$

and

$$\hat{S}_z |S,M_S\rangle = M_S |S,M_S\rangle,\quad M_S = -S, -S + 1, \ldots, S-1, S$$

The spin multiplicity is defined as

$$\text{MULTIPLICITY} = 2S + 1,$$

and this quantity is used instead of $S$ to specify which spin state
is being computed.

Unless otherwise specified, when a psi4 `Molecule` object is created,
the molecular charge is assumed to be 0, and the spin multiplicity
is assumed to be 1 (singlet state).
These quantities may specified by the user in the first line of the
geometry input:
```
    molecule {
      0 1 # <-- charge and multiplicity (neutral,singlet)
      ...
    }

    molecule {
      1 4 # <-- (cation,quartet)
      ...
    }
```
The charge and multiplicity specified in the molecule section of the input (and 
read by psi4) are stored in the `Wavefunction` object generated by psi4
and are read by Forte, where they are used in the rest of the computation.

To select a different charge and multiplicity in Forte, it is possible to specify
the options `CHARGE` and `MULTIPLICITY` in the forte input section.
These options override the value passed in via the `Wavefunction` object
(and the `Wavefunction` object takes priority over the values specified
in the molecular geometry input).

## Specifying the z component of spin ($M_S$)

Most quantum chemistry codes take as an input the spin multiplicity and then
select the highest possible value of $M_S$ compatible with $S$,
that is, $M_S = S$.
Note that $M_S$ is connected to the number of alpha/beta electrons via

$$M_S = \frac{1}{2} (N_\alpha - N_\beta)$$

and, therfore, together with the total number of electrons, it determines the
value of $N_\alpha$ and $N_\beta$.

In Forte, modules will select either the lowest absolute or highest value of $M_S$
compatible with $S$ (as specified via `MULTIPLICITY`), depending on internal
details of the implementation.
For example, if `MULTIPLICITY = 3` and $M_S$ is not specified, the FCI code
in Forte will assume that the user is interested in the solution with
$M_S = 0$.

However, Forte also allows the user to select electronic states with a well defined
value of $M_S$. This quantity may be specified via the option `MS`.
This option is of type `double`, so it should be specified as `0.0`, `-1.5`, etc.

For example, the following input requests the $M_S = 0$ component of a
triplet state:
```
    # triplet, m_s = 0
    set forte {
        multiplicity = 3
        ms = 0.0
    }
```
while the following gives the $M_S = -1$ component:
```
    # triplet, m_s = 1
    set forte {
        multiplicity = 3
        ms = -1.0
    }
```

## Definition of orbital spaces

Running a Forte computation requires specifying a partitioning of the molecular
orbitals. Forte defines five types of elementary orbital spaces:

1. Frozen doubly occupied orbitals (`FROZEN_DOCC`). These orbitals are always
doubly occupied.

2. Restricted doubly occupied orbitals (`RESTRICTED_DOCC`). Orbitals that are
treated as doubly occupied by method for static correlation.
Restricted doubly occupied orbitals are allowed to be excited in
in methods that add dynamic electron correlation.

3. Active/generalized active orbitals (`ACTIVE`/`GASn`).
Used to define active spaces or generalized active spaces for static correlation methods.
These orbitals are partially occupied.
Standard complete active spaces can be specified either via the
`ACTIVE` or the `GAS1` orbital space.
For generalized active spaces, the user must provide the number of orbitals
in each irrep for all the GAS spaces required.
`GAS1` through `GAS6` are currently supported.

4. Restricted unoccupied orbitals (`RESTRICTED_UOCC`). Also called virtuals,
these orbitals are ignored by methods for static correlation but considered by
dynamic correlation approaches.

5. Frozen unoccupied orbitals (`FROZEN_UOCC`). These orbitals are always
unoccupied.

The following table summarizes the properties of these orbital spaces:

| Space           | Occupation in CAS/GAS  | Occupation in correlated methods |  Description  |
|---|---|---|---|
| `FROZEN_DOCC`     |     2      |     2         |  Frozen doubly occupied orbitals     |
| `RESTRICTED_DOCC` |     2      |    0-2        |  Restricted doubly occupied orbitals |
| `GAS1`, `GAS2`, ... |    0-2     |    0-2        |  Generalized active spaces           |
| `RESTRICTED_UOCC` |     0      |    0-2        |  Restricted unoccupied orbitals      |
| `FROZEN_UOCC`     |     0      |     0         |  Frozen unoccupied orbitals          |

**Note**: Forte makes a distinction between elementary and `composite` orbital spaces.
  The spaces defined above are all elementary, except for `ACTIVE`, which is
  defined as the composite space of all the GAS spaces, that is,
  `ACTIVE` = `GAS1 | GAS2 | GAS3 | GAS4 | GAS5 | GAS6`.
  When the user specifies the value of a composite space like `ACTIVE`, then all the
  orbitals are by default assigned to the first space, which in the case of `ACTIVE` is `GAS1`.
  It is important also to note that when there is more than one irrep, the orbitals within a
  composite space are ordered **first** by irrep and then by elementary space.
  This is important to keep in mind when plotting orbitals or for developers writing code in forte.

## Orbital space specification

Selecting the correct set of orbitals for a multireference computation is
perhaps one of the most important steps in setting up an input file.
To specify an orbital space, the user must provide the
number of orbitals contained in each irrep (see Point group symmetry).
Since Forte only supports Abelian groups, each orbital space can be specified
by a vector of integers with at most eight entries.
Note that irreps are arranged according to Cotton's book
(*Chemical Applications of Group Theory*).

The following is an example of a computation on BeH$_2$. This system has 6
electrons. We freeze the Be 1s-like orbital, which has A$_1$ symmetry.
The 2a$_1$ orbital is restricted doubly occupied and the
3a$_1$/1b$_2$ orbitals belong to the active space. The remaining
orbitals belong to the `RESTRICTED_UOCC` set and no virtual orbitals are
frozen:
```
    set forte{
        #          A1 A2 B1 B2
        frozen_docc      [1 ,0 ,0 ,0]
        restricted_docc  [2 ,0 ,0 ,0]
        active           [1 ,0 ,0 ,1]
        restricted_uocc  [4 ,0 ,2 ,3]
        frozen_uocc      [0 ,0 ,0 ,0]
    }
```

## Partial specification of orbital spaces and space priority

Specifying all five orbital spaces for each computation is tedious and error prone.
Forte can help reduce the number of orbital spaces that the user needs to
specify by making certain assumptions.
The class that controls orbital spaces (`MOSpaceInfo`) assumes that orbital
spaces have the following priority:

    GAS1 (= ACTIVE) > RESTRICTED_UOCC > RESTRICTED_DOCC > FROZEN_DOCC > FROZEN_UOCC > GAS2 > ...

When the input does not contain all five orbital spaces, Forte will infer the
size of other orbital spaces. It first sums up all the orbitals specified by
the user, and then assigns any remaining orbitals to the space not specified in
the input that has the highest priority.

In the case of the BeH$_2$ example, it is necessary to specify only the
`FROZEN_DOCC`, `RESTRICTED_DOCC`, and `ACTIVE` orbital spaces:

    set forte{
        frozen_docc        [1 ,0 ,0 ,0]
        restricted_docc    [2 ,0 ,0 ,0]
        active             [1 ,0 ,0 ,1]

        # Forte will automatically assign the following:
        # restricted_uocc  [4 ,0 ,2 ,3]
        # frozen_uocc      [0 ,0 ,0 ,0]
        # gas1             [1 ,0 ,0 ,1]
        # gas2             [0 ,0 ,0 ,0]
        # gas3             [0 ,0 ,0 ,0]
        # gas4             [0 ,0 ,0 ,0]
        # gas5             [0 ,0 ,0 ,0]
        # gas6             [0 ,0 ,0 ,0]
}

the remaining 9 orbitals are automatically assigned to the `RESTRICTED_UOCC`
space. This space, together with `FROZEN_UOCC`, was not specified in the input.
However, `RESTRICTED_UOCC` has higher priority than the `FROZEN_UOCC` space,
so Forte will assign all the remaining orbitals to the `RESTRICTED_UOCC` set.

A Forte input with no orbital space specified will assign all orbitals to the
active space:

    set forte{
        # Forte will automatically assign the following:
        # frozen_docc      [0 ,0 ,0 ,0]
        # restricted_docc  [0 ,0 ,0 ,0]
        # active           [7 ,0 ,2 ,4]
        # restricted_uocc  [0 ,0 ,0 ,0]
        # frozen_uocc      [0 ,0 ,0 ,0]
    }

Note that except for computations with small basis sets,
declaring all orbitals active might be unfeasible.

As a general rule, it is recommended that users run SCF computations and
inspect the orbitals prior to selecting an active space.

## Occupation numbers of GAS wave functions

General active space (GAS) wave functions are defined by partitioning the
active space into subspaces and specifying constraints on the occupation of
these subspaces.
To specify a general active space (GAS) wave function, the user must select the
GAS spaces (see Definition of orbital spaces) and the minimum and maximum
occupation numbers of each GAS space.
This is done by passing two list of integers for each `GASN` space, `GASNMIN` and
`GASNMAX`. For example, the following input defines the orbitals 
associated with two GAS spaces (GAS1 and GAS2).

    set forte{
        gas1       [2,0,0,0]
        gas2       [2,0,1,2]
        gas1min    [2]        
        gas1max    [4]
    }

The options `GAS1MIN` and
`GAS1MAX` specify the minimum and maximum numbers allowed in the GAS1 space.
This information is sufficient to determine all possible GAS occupations.